In [1]:
import keras
from keras_bert import get_custom_objects
from keras_bert import Tokenizer
import numpy as np
import os
import codecs
import pandas as pd

Using TensorFlow backend.


# 塞入前面資料，符合繳交格式
它前面部分不會評分，所以就塞0就好

In [2]:
orderId_scores = []

af = pd.read_csv('task1_sample_submission.csv')
for or_id in af['order_id'][0:131166]:
    orderId_scores.append([or_id,0,0,0,0,0,0])

# 匯入預訓練的model
總共會有六個模型，利用每個模型針對每個句子去做判別

輸入是文本轉乘BERT格式，輸出是該分類之分數。分數只要大於X就判別為該分類

需要再測試每個類別的X需要為多少以上才判別為該類(取最小值?)

In [3]:
background_model  = keras.models.load_model('/home/tn/bert_code/model/background_model.h5', custom_objects=get_custom_objects())
objectives_model  = keras.models.load_model('/home/tn/bert_code/model/objectives_model.h5', custom_objects=get_custom_objects())
methods_model     = keras.models.load_model('/home/tn/bert_code/model/methods_model.h5', custom_objects=get_custom_objects())
results_model     = keras.models.load_model('/home/tn/bert_code/model/results_model.h5', custom_objects=get_custom_objects())
conclusions_model = keras.models.load_model('/home/tn/bert_code/model/conclusions_model.h5', custom_objects=get_custom_objects())





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.









Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




# Tokenize

In [4]:
pretrained_path = '/home/tn/bert_code/cased_L-12_H-768_A-12'
vocab_path = os.path.join(pretrained_path, 'vocab.txt')

token_dict = {}
with codecs.open(vocab_path, 'r') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
        
tokenizer = Tokenizer(token_dict)

# 針對未知句子做預測
先匯入字典，並對句子做bert輸入格式轉換，分別為index與segment

而後將轉換後格式輸入到model，出來的數值即為屬於該分類之分數，越高表示越屬於該分類

In [5]:
df = pd.read_csv('private_data_split.csv')

data_len = len(df['order_id'])

for x in range(data_len):
    
    order_id = df['order_id'][x]
    print(order_id)
    text = df['sentence'][x]

    #轉換成bert輸入格式
    indices, segments = tokenizer.encode(first=text, max_len=100)

    #預測為單一數值
    background_predict  = background_model.predict([np.array([indices]), np.array([segments])])[0]
    objectives_predict  = objectives_model.predict([np.array([indices]), np.array([segments])])[0]
    methods_predict     = methods_model.predict([np.array([indices]), np.array([segments])])[0]
    results_predict     = results_model.predict([np.array([indices]), np.array([segments])])[0]
    conclusions_predict = conclusions_model.predict([np.array([indices]), np.array([segments])])[0]

    background_score  = background_predict[0]
    objectives_score  = objectives_predict[0]
    methods_score     = methods_predict[0]
    results_score     = results_predict[0]
    conclusions_score = conclusions_predict[0]

    #如果數值小於等於0.4填 0，大於0.4則填 1
    all_score = [background_score,objectives_score,methods_score,results_score,conclusions_score]
    
    for i in range(len(all_score)):
        if(all_score[i] > 0.4):
            all_score[i] = 1
        else:
            all_score[i] = 0

    #如果前五個都為 0，則屬於others類別
    if(1 not in all_score):
        all_score.append(1)
    else:
        all_score.append(0)

    #塞入order_id
    all_score.insert(0,order_id)

    orderId_scores.append(all_score)


T20001_S001
T20001_S002
T20001_S003
T20001_S004
T20001_S005
T20001_S006
T20001_S007
T20001_S008
T20002_S001
T20002_S002
T20002_S003
T20002_S004
T20002_S005
T20003_S001
T20003_S002
T20003_S003
T20003_S004
T20003_S005
T20003_S006
T20004_S001
T20004_S002
T20004_S003
T20004_S004
T20004_S005
T20004_S006
T20005_S001
T20005_S002
T20005_S003
T20005_S004
T20005_S005
T20005_S006
T20005_S007
T20006_S001
T20006_S002
T20006_S003
T20006_S004
T20006_S005
T20006_S006
T20006_S007
T20006_S008
T20007_S001
T20007_S002
T20007_S003
T20007_S004
T20007_S005
T20007_S006
T20007_S007
T20007_S008
T20008_S001
T20008_S002
T20008_S003
T20008_S004
T20008_S005
T20008_S006
T20009_S001
T20009_S002
T20009_S003
T20009_S004
T20009_S005
T20009_S006
T20010_S001
T20010_S002
T20010_S003
T20010_S004
T20010_S005
T20010_S006
T20010_S007
T20010_S008
T20010_S009
T20010_S010
T20010_S011
T20010_S012
T20011_S001
T20011_S002
T20011_S003
T20011_S004
T20011_S005
T20011_S006
T20011_S007
T20011_S008
T20012_S001
T20012_S002
T20012_S003
T200

T20106_S002
T20106_S003
T20106_S004
T20106_S005
T20107_S001
T20107_S002
T20107_S003
T20107_S004
T20107_S005
T20107_S006
T20107_S007
T20107_S008
T20108_S001
T20108_S002
T20108_S003
T20108_S004
T20108_S005
T20108_S006
T20108_S007
T20109_S001
T20109_S002
T20109_S003
T20109_S004
T20109_S005
T20109_S006
T20109_S007
T20109_S008
T20109_S009
T20109_S010
T20110_S001
T20110_S002
T20110_S003
T20110_S004
T20110_S005
T20110_S006
T20111_S001
T20111_S002
T20111_S003
T20111_S004
T20111_S005
T20111_S006
T20111_S007
T20111_S008
T20111_S009
T20111_S010
T20111_S011
T20111_S012
T20112_S001
T20112_S002
T20112_S003
T20112_S004
T20112_S005
T20112_S006
T20113_S001
T20113_S002
T20113_S003
T20113_S004
T20113_S005
T20113_S006
T20113_S007
T20113_S008
T20114_S001
T20114_S002
T20114_S003
T20114_S004
T20114_S005
T20114_S006
T20114_S007
T20115_S001
T20115_S002
T20115_S003
T20115_S004
T20115_S005
T20116_S001
T20116_S002
T20116_S003
T20116_S004
T20116_S005
T20116_S006
T20116_S007
T20116_S008
T20117_S001
T20117_S002
T201

T20210_S008
T20211_S001
T20211_S002
T20211_S003
T20211_S004
T20211_S005
T20211_S006
T20211_S007
T20211_S008
T20211_S009
T20212_S001
T20212_S002
T20212_S003
T20212_S004
T20212_S005
T20212_S006
T20213_S001
T20213_S002
T20213_S003
T20213_S004
T20213_S005
T20213_S006
T20213_S007
T20213_S008
T20214_S001
T20214_S002
T20214_S003
T20215_S001
T20215_S002
T20215_S003
T20216_S001
T20216_S002
T20216_S003
T20217_S001
T20217_S002
T20217_S003
T20217_S004
T20217_S005
T20217_S006
T20217_S007
T20218_S001
T20218_S002
T20218_S003
T20219_S001
T20219_S002
T20219_S003
T20219_S004
T20220_S001
T20220_S002
T20220_S003
T20220_S004
T20220_S005
T20220_S006
T20220_S007
T20220_S008
T20221_S001
T20221_S002
T20221_S003
T20221_S004
T20221_S005
T20221_S006
T20221_S007
T20221_S008
T20221_S009
T20222_S001
T20222_S002
T20222_S003
T20222_S004
T20223_S001
T20223_S002
T20223_S003
T20223_S004
T20224_S001
T20224_S002
T20224_S003
T20224_S004
T20224_S005
T20224_S006
T20225_S001
T20225_S002
T20225_S003
T20226_S001
T20226_S002
T202

T20314_S003
T20314_S004
T20314_S005
T20314_S006
T20314_S007
T20314_S008
T20314_S009
T20315_S001
T20315_S002
T20315_S003
T20315_S004
T20315_S005
T20316_S001
T20316_S002
T20316_S003
T20316_S004
T20316_S005
T20316_S006
T20316_S007
T20316_S008
T20316_S009
T20317_S001
T20317_S002
T20317_S003
T20317_S004
T20318_S001
T20318_S002
T20318_S003
T20318_S004
T20318_S005
T20318_S006
T20319_S001
T20319_S002
T20319_S003
T20319_S004
T20319_S005
T20319_S006
T20319_S007
T20319_S008
T20320_S001
T20320_S002
T20320_S003
T20320_S004
T20320_S005
T20321_S001
T20321_S002
T20321_S003
T20321_S004
T20321_S005
T20321_S006
T20321_S007
T20321_S008
T20322_S001
T20322_S002
T20322_S003
T20322_S004
T20322_S005
T20323_S001
T20323_S002
T20323_S003
T20323_S004
T20323_S005
T20323_S006
T20324_S001
T20324_S002
T20324_S003
T20324_S004
T20324_S005
T20325_S001
T20325_S002
T20325_S003
T20326_S001
T20326_S002
T20326_S003
T20326_S004
T20326_S005
T20326_S006
T20327_S001
T20327_S002
T20327_S003
T20327_S004
T20327_S005
T20327_S006
T203

T20419_S002
T20419_S003
T20419_S004
T20419_S005
T20420_S001
T20420_S002
T20420_S003
T20420_S004
T20420_S005
T20420_S006
T20420_S007
T20420_S008
T20420_S009
T20421_S001
T20421_S002
T20421_S003
T20421_S004
T20421_S005
T20421_S006
T20421_S007
T20421_S008
T20421_S009
T20422_S001
T20422_S002
T20422_S003
T20422_S004
T20422_S005
T20422_S006
T20422_S007
T20422_S008
T20422_S009
T20423_S001
T20423_S002
T20423_S003
T20423_S004
T20423_S005
T20423_S006
T20423_S007
T20424_S001
T20424_S002
T20424_S003
T20424_S004
T20424_S005
T20425_S001
T20425_S002
T20425_S003
T20425_S004
T20425_S005
T20425_S006
T20425_S007
T20426_S001
T20426_S002
T20426_S003
T20426_S004
T20426_S005
T20426_S006
T20426_S007
T20426_S008
T20426_S009
T20427_S001
T20427_S002
T20427_S003
T20427_S004
T20428_S001
T20428_S002
T20428_S003
T20428_S004
T20428_S005
T20428_S006
T20429_S001
T20429_S002
T20429_S003
T20429_S004
T20429_S005
T20430_S001
T20430_S002
T20430_S003
T20430_S004
T20430_S005
T20430_S006
T20430_S007
T20431_S001
T20431_S002
T204

...

T39518_S002
T39518_S003
T39518_S004
T39518_S005
T39518_S006
T39519_S001
T39519_S002
T39519_S003
T39519_S004
T39520_S001
T39520_S002
T39520_S003
T39520_S004
T39520_S005
T39521_S001
T39521_S002
T39521_S003
T39521_S004
T39521_S005
T39522_S001
T39522_S002
T39522_S003
T39522_S004
T39522_S005
T39522_S006
T39522_S007
T39522_S008
T39522_S009
T39522_S010
T39523_S001
T39523_S002
T39523_S003
T39523_S004
T39523_S005
T39523_S006
T39524_S001
T39524_S002
T39524_S003
T39524_S004
T39524_S005
T39524_S006
T39524_S007
T39524_S008
T39525_S001
T39525_S002
T39525_S003
T39525_S004
T39526_S001
T39526_S002
T39526_S003
T39526_S004
T39526_S005
T39526_S006
T39526_S007
T39526_S008
T39527_S001
T39527_S002
T39527_S003
T39527_S004
T39527_S005
T39527_S006
T39527_S007
T39527_S008
T39528_S001
T39528_S002
T39528_S003
T39528_S004
T39528_S005
T39528_S006
T39528_S007
T39528_S008
T39528_S009
T39529_S001
T39529_S002
T39529_S003
T39529_S004
T39529_S005
T39530_S001
T39530_S002
T39530_S003
T39530_S004
T39530_S005
T39530_S006
T395

T39621_S007
T39621_S008
T39621_S009
T39621_S010
T39622_S001
T39622_S002
T39622_S003
T39622_S004
T39622_S005
T39622_S006
T39622_S007
T39622_S008
T39622_S009
T39623_S001
T39623_S002
T39623_S003
T39623_S004
T39623_S005
T39623_S006
T39623_S007
T39623_S008
T39623_S009
T39623_S010
T39623_S011
T39624_S001
T39624_S002
T39624_S003
T39624_S004
T39624_S005
T39624_S006
T39624_S007
T39625_S001
T39625_S002
T39625_S003
T39625_S004
T39625_S005
T39625_S006
T39626_S001
T39626_S002
T39626_S003
T39627_S001
T39627_S002
T39627_S003
T39627_S004
T39627_S005
T39627_S006
T39628_S001
T39628_S002
T39628_S003
T39628_S004
T39628_S005
T39628_S006
T39628_S007
T39629_S001
T39629_S002
T39629_S003
T39629_S004
T39629_S005
T39629_S006
T39629_S007
T39630_S001
T39630_S002
T39630_S003
T39630_S004
T39630_S005
T39630_S006
T39630_S007
T39630_S008
T39631_S001
T39631_S002
T39631_S003
T39631_S004
T39631_S005
T39631_S006
T39631_S007
T39631_S008
T39631_S009
T39631_S010
T39631_S011
T39632_S001
T39632_S002
T39632_S003
T39632_S004
T396

T39723_S002
T39723_S003
T39723_S004
T39723_S005
T39723_S006
T39723_S007
T39723_S008
T39723_S009
T39723_S010
T39723_S011
T39723_S012
T39724_S001
T39724_S002
T39724_S003
T39724_S004
T39724_S005
T39725_S001
T39725_S002
T39725_S003
T39725_S004
T39725_S005
T39725_S006
T39725_S007
T39725_S008
T39725_S009
T39725_S010
T39725_S011
T39726_S001
T39726_S002
T39726_S003
T39726_S004
T39726_S005
T39726_S006
T39726_S007
T39726_S008
T39727_S001
T39727_S002
T39727_S003
T39727_S004
T39727_S005
T39727_S006
T39728_S001
T39728_S002
T39728_S003
T39728_S004
T39728_S005
T39728_S006
T39728_S007
T39728_S008
T39729_S001
T39729_S002
T39729_S003
T39729_S004
T39729_S005
T39729_S006
T39730_S001
T39730_S002
T39730_S003
T39730_S004
T39730_S005
T39730_S006
T39730_S007
T39731_S001
T39731_S002
T39731_S003
T39731_S004
T39731_S005
T39731_S006
T39731_S007
T39732_S001
T39732_S002
T39732_S003
T39732_S004
T39732_S005
T39732_S006
T39732_S007
T39732_S008
T39733_S001
T39733_S002
T39733_S003
T39733_S004
T39733_S005
T39733_S006
T397

T39825_S004
T39825_S005
T39825_S006
T39826_S001
T39826_S002
T39826_S003
T39826_S004
T39826_S005
T39826_S006
T39826_S007
T39826_S008
T39826_S009
T39827_S001
T39827_S002
T39827_S003
T39827_S004
T39827_S005
T39828_S001
T39828_S002
T39828_S003
T39828_S004
T39828_S005
T39828_S006
T39828_S007
T39828_S008
T39829_S001
T39829_S002
T39829_S003
T39829_S004
T39829_S005
T39829_S006
T39829_S007
T39830_S001
T39830_S002
T39830_S003
T39830_S004
T39830_S005
T39830_S006
T39831_S001
T39831_S002
T39831_S003
T39831_S004
T39831_S005
T39831_S006
T39831_S007
T39831_S008
T39832_S001
T39832_S002
T39833_S001
T39833_S002
T39833_S003
T39833_S004
T39833_S005
T39833_S006
T39833_S007
T39833_S008
T39833_S009
T39834_S001
T39834_S002
T39834_S003
T39834_S004
T39834_S005
T39834_S006
T39834_S007
T39835_S001
T39835_S002
T39835_S003
T39836_S001
T39836_S002
T39836_S003
T39836_S004
T39836_S005
T39837_S001
T39837_S002
T39837_S003
T39837_S004
T39837_S005
T39837_S006
T39837_S007
T39837_S008
T39838_S001
T39838_S002
T39838_S003
T398

T39925_S001
T39926_S001
T39926_S002
T39926_S003
T39926_S004
T39926_S005
T39926_S006
T39926_S007
T39926_S008
T39926_S009
T39926_S010
T39926_S011
T39927_S001
T39927_S002
T39927_S003
T39927_S004
T39927_S005
T39927_S006
T39927_S007
T39927_S008
T39927_S009
T39927_S010
T39928_S001
T39928_S002
T39928_S003
T39928_S004
T39928_S005
T39928_S006
T39929_S001
T39929_S002
T39929_S003
T39929_S004
T39929_S005
T39929_S006
T39929_S007
T39929_S008
T39929_S009
T39930_S001
T39930_S002
T39931_S001
T39931_S002
T39931_S003
T39931_S004
T39931_S005
T39932_S001
T39932_S002
T39932_S003
T39932_S004
T39932_S005
T39932_S006
T39932_S007
T39932_S008
T39933_S001
T39933_S002
T39933_S003
T39933_S004
T39933_S005
T39933_S006
T39933_S007
T39933_S008
T39933_S009
T39933_S010
T39934_S001
T39934_S002
T39934_S003
T39934_S004
T39934_S005
T39934_S006
T39935_S001
T39935_S002
T39935_S003
T39935_S004
T39935_S005
T39935_S006
T39935_S007
T39936_S001
T39936_S002
T39936_S003
T39936_S004
T39936_S005
T39936_S006
T39936_S007
T39936_S008
T399

In [7]:
import csv
with open("./score_private_data.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["order_id","BACKGROUND","OBJECTIVES","METHODS","RESULTS","CONCLUSIONS","OTHERS"])
    writer.writerows(orderId_scores)